In [2]:
from model import Deeplabv3
import pickle
import cv2
import numpy as np
import keras
import matplotlib 
import matplotlib.image as mp
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam
from keras import backend as K
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import *

H:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
H:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
H:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
H:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '

In [3]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    y_pred = y_pred[:,:,:,1]
    y_true = y_true[:,:,:,1]
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:

def dice_coefficient(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coefficient_loss(y_true, y_pred):
    return (1 - dice_coefficient(y_true, y_pred))#-dice_coefficient(y_true, y_pred)

In [5]:
def load_data():
    f = open('data.pkl','rb')
    pkl = pickle.load(f)
    f.close()
    data_o = pkl["data"]
    label_o = pkl["label"]
    f_new = open('data_new.pkl','rb')
    pkl = pickle.load(f_new)
    f_new.close()
    data_o_new = pkl["data"]
    label_o_new = pkl["label"]
    return np.concatenate([data_o,data_o_new]), np.concatenate([label_o, label_o_new])
# label_o = keras.utils.to_cat

In [6]:
adam = Adam(lr=0.0001)
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)

data_o, label_o = load_data()

Instructions for updating:
Colocations handled automatically by placer.


UnpicklingError: pickle data was truncated

In [6]:
label_o = keras.utils.to_categorical(label_o)

In [7]:
data_o = np.concatenate([data_o[:,0:256,0:256,:],data_o[:,0:256,256:,:],data_o[:,256:,0:256,:],data_o[:,256:,256:,:]])

In [9]:
data_o.shape

(13496, 256, 256, 3)

In [10]:
label_o = np.concatenate([label_o[:,0:256,0:256,:],label_o[:,0:256,256:,:],label_o[:,256:,0:256,:],label_o[:,256:,256:,:]])

In [ ]:
label_o.shape

In [11]:
batch_size = 6
SEED = 42
def my_generator(x_train, y_train, batch_size):
    data_generator = keras.preprocessing.image.ImageDataGenerator(
#                         rescale=1./255,
                        featurewise_center=True,
                       samplewise_center=True,
                       featurewise_std_normalization=True,
                       samplewise_std_normalization=True,
                        brightness_range=[0.5,1.5], 
                        vertical_flip = True,
#                         zoom_range = [1,1.5],
                        horizontal_flip=True).flow(x_train, x_train, batch_size, seed=SEED)
    mask_generator = keras.preprocessing.image.ImageDataGenerator(
#                         rescale=1./255,
#                         brightness_range=[0.2,1.3], 
                        vertical_flip = True,
#                         zoom_range = [1,1.5],
                         fill_mode='nearest',
                        horizontal_flip=True).flow(y_train, y_train, batch_size, seed=SEED)
    while True:
        x_batch, _ = data_generator.next()
        y_batch, _ = mask_generator.next()
        yield x_batch, y_batch

In [1]:
print("done!")
print("training model")
check_point_file = "./deeplabv3_model_256*256_new.h5"
model_checkpoint = ModelCheckpoint(check_point_file, monitor="val_loss", save_best_only=True,
                          save_weights_only=False, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor=0.5,
            patience=5, min_lr=1e-6)
model = Deeplabv3(input_shape=(256,256,3), classes=2)
# model = get_model(input_shape=(512,512,3))

done!
training model


NameError: name 'ModelCheckpoint' is not defined

In [14]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 128, 128, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 128, 128, 32) 128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 128, 128, 32) 0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

In [16]:
from sklearn.metrics import f1_score,recall_score, precision_score
class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))[:,:,:,0]).round().flatten()
        val_targ = self.validation_data[1][:,:,:,0].flatten()
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print('- val_f1: %.4f - val_precision: %.4f - val_recall: %.4f'%(_val_f1, _val_precision, _val_recall))
        return

In [ ]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [ ]:
from sklearn.utils import class_weight

In [17]:
# weights_loss = weighted_categorical_crossentropy([38,0.5])
f1 = Metrics()
callbacks=[model_checkpoint, f1, reduce_lr]
par_model = keras.utils.multi_gpu_model(model,gpus=2)

par_model.compile(optimizer = Adam(lr=7e-4, epsilon=1e-8, decay=1e-6), 
              loss = 
#                  weights_loss, 
              keras.losses.categorical_crossentropy,
#               dice_coefficient_loss,
              metrics=['accuracy'])
# par_model.load_weights(check_point_file)

In [ ]:
data_o.shape

In [19]:
history = par_model.fit(data_o, label_o,
                    epochs=10, 
                    batch_size=24,
#                     steps_per_epoch = 3000,
                    callbacks=callbacks,
                    validation_split=0.05,
                    verbose=1)

Train on 12821 samples, validate on 675 samples
Epoch 1/10
12821/12821 [==============================] - 316s 25ms/step - loss: 0.0455 - acc: 0.9871 - val_loss: 0.0369 - val_acc: 0.9877

Epoch 00001: val_loss improved from inf to 0.03692, saving model to ./deeplabv3_model_256*256_new.h5
- val_f1: 0.9938 - val_precision: 0.9877 - val_recall: 1.0000
Epoch 2/10
12821/12821 [==============================] - 318s 25ms/step - loss: 0.0289 - acc: 0.9871 - val_loss: 0.0559 - val_acc: 0.9877

Epoch 00002: val_loss did not improve from 0.03692
- val_f1: 0.9938 - val_precision: 0.9877 - val_recall: 1.0000
Epoch 3/10
12821/12821 [==============================] - 321s 25ms/step - loss: 0.0257 - acc: 0.9896 - val_loss: 0.0419 - val_acc: 0.9885

Epoch 00003: val_loss did not improve from 0.03692
- val_f1: 0.9942 - val_precision: 0.9928 - val_recall: 0.9955
Epoch 4/10
12821/12821 [==============================] - 322s 25ms/step - loss: 0.0231 - acc: 0.9913 - val_loss: 0.0232 - val_acc: 0.9919

Epo

KeyboardInterrupt: 

In [ ]:
# history = par_model.fit_generator(my_generator(data_o, label_o, 8),
#                     epochs=200, 
#                     steps_per_epoch = 3000,
# #                     callbacks=callbacks,
# #                     validation_split=0.05,
#                     verbose=1)

In [ ]:
import PIL
from matplotlib import pyplot as plt

In [ ]:
model.save_weights(check_point_file)

In [ ]:
%matplotlib inline

In [ ]:
i=6
pred = model.predict(data_o[i:i+1])
plt.figure(figsize=(10,20))
plt.subplot(121)
plt.imshow(label_o[i,:,:,1])
pred = np.argmax(pred, axis=-1)[0]
# PIL.Image.fromarray(label_o[-1,:,:,1].astype(np.uint8)*255)
plt.subplot(122)
plt.imshow(pred)